### Question 2: - [20 marks]                

##### Consider a continuous attribute in your dataset as the target variable, perform regression analysis using different ensemble methods, visualize and interpret the results. 
Student Name - Swaraj Shaw  
Email Address - 10592394@mydbs.ie  
Module Name - B9AI102 Machine Learning and Pattern Recognition (B9AI102_2122_TMD2)  
Lecturer(s)Name: Dr Shahram Azizi

In [1]:
#Load dataset from existing dataset folder
import os
data = './weather-dataset-rattle-package'
os.listdir(data)

['weatherAUS.csv']

In [2]:
train_csv = data + '/weatherAUS.csv'
train_csv

'./weather-dataset-rattle-package/weatherAUS.csv'

In [3]:
#Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

In [4]:
df = pd.read_csv(train_csv)
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
# Dropping the rows that have missing values in the columns 'RainToday' and 'RainTomorrow'.
df.dropna(subset=['RainToday','RainTomorrow'], inplace=True)

In [6]:
#Scatter plot for MaxTemp vs MinTemp by RainToday
px.scatter(df.sample(5000), title='Temperature Trend (MaxTemp vs MinTemp)',
                        x='MinTemp',
                        y='MaxTemp',
                        color='RainToday')

In [7]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [8]:
#Creating a dataframe with all the columns except MaxTemp and then creating a series with only the MaxTemp column.
X = df[['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday','RainTomorrow']]
y = df['MaxTemp']

In [9]:
n_cols = X.select_dtypes(include=np.number).columns.to_list()
c_cols = X.select_dtypes('object').columns.to_list()

In [10]:
n_cols, c_cols

(['MinTemp',
  'MaxTemp',
  'Rainfall',
  'Evaporation',
  'Sunshine',
  'WindGustSpeed',
  'WindSpeed9am',
  'WindSpeed3pm',
  'Humidity9am',
  'Humidity3pm',
  'Pressure9am',
  'Pressure3pm',
  'Cloud9am',
  'Cloud3pm',
  'Temp9am',
  'Temp3pm'],
 ['RainToday', 'RainTomorrow'])

In [11]:
#Replacing the missing values with the mean of the column.
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(df[n_cols])
df[n_cols]=imputer.transform(df[n_cols])

In [12]:
#Missing Values in each column
df[n_cols].isna().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
dtype: int64

In [13]:
#The above code is scaling the data in the range of 0 to 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[n_cols])
df[n_cols]=scaler.transform(df[n_cols])

In [14]:
#Implementing the OneHotEncoder 
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit_transform(df[c_cols])
df[c_cols]

,RainToday,RainTomorrow
0,No,No
1,No,No
2,No,No
3,No,No
4,No,No
...,...,...
145454,No,No
145455,No,No
145456,No,No
145457,No,No


In [15]:
encoder.categories_

[array(['No', 'Yes'], dtype=object), array(['No', 'Yes'], dtype=object)]

In [16]:
encoded_cols = list(encoder.get_feature_names_out(c_cols))
encoded_cols

['RainToday_No', 'RainToday_Yes', 'RainTomorrow_No', 'RainTomorrow_Yes']

In [17]:
df[encoded_cols] = encoder.transform(df[c_cols])

In [18]:
X = df[n_cols + encoded_cols]
y = df['MaxTemp']

In [19]:
X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday_No,RainToday_Yes,RainTomorrow_No,RainTomorrow_Yes
0,0.516509,0.523629,0.001617,0.037741,0.526244,0.294574,0.153846,0.275862,0.71,0.22,0.449587,0.4800,0.888889,0.499917,0.508439,0.522073,1.0,0.0,1.0,0.0
1,0.375000,0.565217,0.000000,0.037741,0.526244,0.294574,0.030769,0.252874,0.44,0.25,0.497521,0.4912,0.492351,0.499917,0.514768,0.570058,1.0,0.0,1.0,0.0
2,0.504717,0.576560,0.000000,0.037741,0.526244,0.310078,0.146154,0.298851,0.38,0.30,0.447934,0.5056,0.492351,0.222222,0.594937,0.548944,1.0,0.0,1.0,0.0
3,0.417453,0.620038,0.000000,0.037741,0.526244,0.139535,0.084615,0.103448,0.45,0.16,0.613223,0.5712,0.492351,0.499917,0.533755,0.612284,1.0,0.0,1.0,0.0
4,0.613208,0.701323,0.002695,0.037741,0.526244,0.271318,0.053846,0.229885,0.82,0.33,0.500826,0.4624,0.777778,0.888889,0.527426,0.673704,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,0.283019,0.502836,0.000000,0.037741,0.526244,0.193798,0.115385,0.149425,0.59,0.27,0.730579,0.7056,0.492351,0.499917,0.350211,0.504798,1.0,0.0,1.0,0.0
145455,0.266509,0.533081,0.000000,0.037741,0.526244,0.193798,0.100000,0.126437,0.51,0.24,0.728926,0.6912,0.492351,0.499917,0.364979,0.533589,1.0,0.0,1.0,0.0
145456,0.285377,0.568998,0.000000,0.037741,0.526244,0.124031,0.100000,0.103448,0.56,0.21,0.710744,0.6720,0.492351,0.499917,0.381857,0.573896,1.0,0.0,1.0,0.0
145457,0.327830,0.599244,0.000000,0.037741,0.526244,0.240310,0.069231,0.103448,0.53,0.24,0.669421,0.6352,0.492351,0.499917,0.415612,0.604607,1.0,0.0,1.0,0.0


In [20]:
y

0         0.523629
1         0.565217
2         0.576560
3         0.620038
4         0.701323
            ...   
145454    0.502836
145455    0.533081
145456    0.568998
145457    0.599244
145458    0.601134
Name: MaxTemp, Length: 140787, dtype: float64

In [21]:
from sklearn.model_selection import train_test_split
#ML Models
from sklearn.linear_model import LinearRegression

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
#Tree
from sklearn.tree import DecisionTreeRegressor
# Ensembling Techniques
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor, BaggingRegressor, AdaBoostRegressor 

#Scores & Metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,recall_score,classification_report,confusion_matrix

In [22]:
# The above code is splitting the data into training and testing data.
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=30)

In [23]:
#Feature Selection using F_Classif
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, SelectFromModel, RFE
selector = SelectKBest(f_classif, k=10)
X_train_reduced = selector.fit_transform(X_train, y_train)
X_train_reduced.shape
print(selector.get_support())

[ True  True False  True  True False False False  True  True  True  True
 False False  True  True False False False False]


In [24]:
#Fitting the model with the reduced features and then predicting the values using Linear Regression.
model_reg = LinearRegression()
model_reg.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test)
predict = model_reg.predict(X_test_reduced)
print('Intercept is :',model_reg.intercept_)
print('Coefficient is :' ,model_reg.coef_)
print('Linear Regression Model Score -',model_reg.score(X_test_reduced,y_test))

Intercept is : 1.1102230246251565e-16
Coefficient is : [ 6.94604507e-16  1.00000000e+00  6.45317133e-16 -2.28983499e-16
  1.38777878e-16 -1.85615412e-16 -5.55111512e-17 -5.55111512e-17
  4.44089210e-16  2.22044605e-16]
Linear Regression Model Score - 1.0


In [25]:
#Error
np.mean((predict-y_test)**2)

1.9300399355214865e-32

In [26]:
#Actual vs Predicted
pd.DataFrame({'actual':y_test,
            'prediction':predict,
            'diff':(y_test-predict)})

,actual,prediction,diff
92683,0.620038,0.620038,0.000000e+00
51737,0.357278,0.357278,1.665335e-16
121194,0.684310,0.684310,2.220446e-16
77331,0.304348,0.304348,0.000000e+00
58070,0.385633,0.385633,1.110223e-16
...,...,...,...
36030,0.669187,0.669187,1.110223e-16
55187,0.465028,0.465028,1.110223e-16
86903,0.574669,0.574669,-1.110223e-16
112302,0.485822,0.485822,1.665335e-16


In [27]:
#Decision Tree
dt_reg = DecisionTreeRegressor()
dt_reg.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test)
predict = dt_reg.predict(X_test_reduced)
# print('Intercept is :',dt_reg.intercept_)
# print('Coefficient is :' ,dt_reg.coef_)
print('Decision Tree Regressor Model Score -',dt_reg.score(X_test_reduced,y_test))

Decision Tree Regressor Model Score - 0.9999998184140426


In [28]:
#SupportVectorRegressor
sv_reg = SVR()
sv_reg.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test)
predict = dt_reg.predict(X_test_reduced)
print('Support Vector Regressor Model Score -',sv_reg.score(X_test_reduced,y_test))

Support Vector Regressor Model Score - 0.9473909592589101


In [29]:
#KNN Regressor
knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test)
predict = knn_reg.predict(X_test_reduced)
print('KNN Regressor Model Score -',knn_reg.score(X_test_reduced,y_test))

KNN Regressor Model Score - 0.9880208924544844


In [31]:
# Random Forest Regressor
cart = RandomForestRegressor()
cart.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test)
predict = cart.predict(X_test_reduced)
print('Random Forest Regressor Model Score -',cart.score(X_test_reduced,y_test))

Random Forest Regressor Model Score - 0.9999998784470586


In [33]:
#AdaBoost Regressor
ada_boost = AdaBoostRegressor(base_estimator=LinearRegression(), n_estimators=450)
ada_boost.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test)
predict = ada_boost.predict(X_test_reduced)
print('AdaBoost Regressor Model Score -',ada_boost.score(X_test_reduced,y_test))

AdaBoost Regressor Model Score - 1.0


**Linear Regression** stands out as the best model following by adaboost which is typically the same as it using Liner Regression as the base model.
| Models                  | Accuracy  |
|-------------------------|-----------|
| Linear Regression       | 1.0       |
| Decision Tree           | 0.9999    |
| Support Vector Regressor| 0.9473    |
| KNN Regressor           | 0.9880    |
| Random Forest Regressor | 0.9999    |
| AdaBoost Regressor      | 1.0       |

In [34]:
# # HyperParam Tuning using RandomSearchCV
# from sklearn.model_selection import RandomizedSearchCV
# import time
# # Decision Tree
# start = time.process_time()
# dt_tune = DecisionTreeRegressor()
# rv_params = {
#     'criterion' : ['squared_error', 'friedman_mse','absolute_error', 'poisson'],
#     'max_depth' : [3, 5, 7, 10],
#     'min_samples_split' : range(2, 10, 1),
#     'min_samples_leaf' : range(2, 10, 1)
# }

# rs_search = RandomizedSearchCV(dt_tune, rv_params, cv = 10, n_jobs = -1, verbose = 1, random_state=42)
# rs_search.fit(X_train_reduced, y_train)
# print(time.process_time() - start)

# # best parameters and best score

# print(rs_search.best_params_)
# print(rs_search.best_score_)

In [35]:
# import time
# start = time.process_time()
# #RandomForest Classifier
# rand_clf = RandomForestRegressor(n_jobs=-1)

# rand_params = {'max_depth': [10, 20, 30, 40, 50, None],
#              'criterion':['squared_error','absolute_error', 'poisson'],
#              'min_samples_leaf':[1, 2, 3, 4, 5],
#              'min_samples_split': [2, 5, 7, 9, 11],
#              'max_features': ['auto', 'sqrt','log2'],
#              'n_estimators': [100, 150],
#               }


    
# rand_rs = RandomizedSearchCV(rand_clf, rand_params, cv=10, n_jobs = -1, random_state=42)
# rand_rs.fit(X_train_reduced, y_train)

# print(time.process_time() - start)
# print(rand_rs.best_params_)
# print(rand_rs.best_score_)

### Stacking Models

In [1]:
# def get_stacking():
# 	# define the base models: level0 models
# 	level0 = list()
# 	level0.append(('linear_reg', LinearRegression()))
# 	level0.append(('cart', RandomForestRegressor(n_estimators=450, criterion='entropy')))
# 	level0.append(('svm', SVR(kernel='rbf')))
# 	level0.append(('knn', KNeighborsRegressor()))
# 	# define meta learner model: level 1
# 	level1 = LinearRegression()
# 	# define the stacking ensemble
# 	model = StackingRegressor(estimators=level0, final_estimator=level1, cv=10)
# 	return model

In [ ]:
# # get a list of models to evaluate
# def get_models():
#   models = dict()
#   models['linear_reg']=LinearRegression()
#   models['svc_lr']=SVR(kernel='linear')
#   models['svm_polly'] = SVR(kernel='poly')
#   models['svm_rbf'] = SVR(kernel='rbf')
#   models['svm_sigmoid'] = SVR(kernel='sigmoid')
#   models['knn']= KNeighborsRegressor()

#   models['dt_ent'] = DecisionTreeRegressor(min_samples_split= 6, min_samples_leaf= 8, max_depth= 10, criterion='entropy')
#   models['dt_gini'] = DecisionTreeRegressor(min_samples_split= 6, min_samples_leaf= 8, max_depth= 10, criterion='gini')
#   models['dt_gini'] = DecisionTreeRegressor(min_samples_split= 6, min_samples_leaf= 8, max_depth= 10, criterion='squared_error')
#   models['rf'] = RandomForestRegressor(n_estimators=450, criterion='entropy')
#   models['bagg_dt'] = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=450)
#   models['bagg_svr'] = BaggingRegressor(base_estimator=SVR(), n_estimators=450)
#   models['bagg_knn'] = BaggingRegressor(base_estimator=KNeighborsRegressor(), n_estimators=450)
#   models['adaboost'] = AdaBoostRegressor(base_estimator=LinearRegression(), n_estimators=450)
#   models['stacking']= get_stacking()
#   return models


In [ ]:
# def evaluate_model_cross(model, X_train, y_test):
# 	cv = 10
# 	scores = cross_val_score(model, X_train, y_test, scoring='neg_mean_absolute_error', cv=cv,  n_jobs=-1, error_score='raise')
# 	return scores